## This is a cell to allow users to document the logic
It will not be used when creating the Arc job.

In [ ]:
{
  "type": "DelimitedExtract",
  "name": "extract data from green_tripdata/0",
  "environments": [
    "production",
    "test"
  ],
  "inputURI": ${ETL_JOB_ROOT}"/data/input/green_tripdata/0/green_tripdata_2013-08.csv",
  "outputView": "green_tripdata0_raw",
  "persist": false,
  "delimiter": "Comma",
  "quote": "DoubleQuote",
  "header": true,
  "authentication": {},
  "params": {}
}

In [ ]:
%%sql
SELECT * 
FROM green_tripdata0_raw

In [ ]:
{
  "type": "TypingTransform",
  "name": "apply green_tripdata data types",
  "environments": [
    "production",
    "test"
  ],
  "inputURI": ${ETL_JOB_ROOT}"/meta/green_tripdata/0/green_tripdata.json",
  "inputView": "green_tripdata0_raw",
  "outputView": "green_tripdata0",
  "persist": true,
  "authentication": {},
  "params": {}
}

In [ ]:
{
  "type": "SQLValidate",
  "name": "ensure no errors exist after data typing",
  "environments": [
    "production",
    "test"
  ],
  "inputURI": ${ETL_JOB_ROOT}"/job/0/sqlvalidate_errors.sql",
  "sqlParams": {
    "table_name": "green_tripdata0"
  },
  "authentication": {},
  "params": {}
}

In [ ]:
{
  "type": "SQLTransform",
  "name": "enrich dataset by adding distance",
  "environments": [
    "production",
    "test"
  ],
  "inputURI": ${ETL_JOB_ROOT}"/job/1/enrich_greentripdata0.sql",
  "outputView": "green_tripdata0_enriched",
  "persist": true,
  "authentication": {},
  "sqlParams": {},
  "params": {}
}

In [ ]:
{
  "type": "MLTransform",
  "name": "apply machine learning model",
  "environments": [
    "production",
    "test"
  ],
  "inputURI": ${ETL_JOB_ROOT}"/job/1/green_tripdata_enriched.model",
  "inputView": "green_tripdata0_enriched",
  "outputView": "green_tripdata0_prediction",
  "persist": false,
  "authentication": {},
  "params": {}
}

In [ ]:
{
  "type": "ParquetLoad",
  "name": "write green_tripdata0_prediction records to parquet",
  "environments": [
    "production",
    "test"
  ],
  "inputView": "green_tripdata0_prediction",
  "outputURI": ${ETL_JOB_ROOT}"/data/output/green_tripdata0_prediction.parquet",
  "numPartitions": 4,
  "partitionBy": [],
  "authentication": {},
  "saveMode": "Overwrite",
  "params": {}
}